<a href="https://colab.research.google.com/github/starryesh22/Google_Colab/blob/Heartbeat/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os   # os 모듈은 운영 체제에 종속된 기능을 사용할 수 있게 해주는 기능을 제공. 예 : 파일 시스템에 읽기 또는 쓰기 가능.

import torch  # torch 모듈은 텐서를 다루고 딥러닝 모델을 구현하는 데 사용되는 라이브러리

from torch.utils.data import Dataset  # torch.utils.data 모듈은 PyTorch에서 데이터셋과 데이터 로더를 다루는 데 사용되는 도구를 제공. Dataset과 같은 클래스를 포함하고 있어서 특정한 요구에 맞는 사용자 정의 데이터셋을 만들 수 있음.

import pandas as pd  # pandas 라이브러리는 데이터 조작과 분석을 위한 라이브러리입니다. 데이터프레임(DataFrame)과 같은 데이터 구조를 제공하여 효율적인 데이터 처리 가능.

import numpy as np  # numpy 라이브러리는 Python에서 과학적인 계산을 위한 기본 패키지. (큰 다차원 배열 및 배열에 대한 수학적인 함수를 지원)

import wfdb  # wfdb는 PhysioNet의 데이터 형식인 WFDB(파형 데이터베이스) 형식을 다루기 위해 설계된 라이브러리



In [ ]:

#1. scaling function  # scaling 함수 정의 ; 주어진 행렬 X에 스케일링을 적용, 스케일링 요소에는 정규 분포에서 추출한 난수가 추가되는데, 평균값은 1.0이고 표준 편차는 sigma임.

# 노이즈는 X의 각 특성에 대해 생성되며 해당 값을 요소별로 조정하는 데 사용


def scaling(X, sigma=0.1):  


    scalingFactor = np.random.normal(loc=1.0, scale=sigma, size=(1, X.shape[1]))
    myNoise = np.matmul(np.ones((X.shape[0], 1)), scalingFactor)
    return X * myNoise  

''' 이 코드는 scaling이라는 함수를 정의하고, 이 함수는 X라는 입력 행렬과 선택적 매개변수인 sigma를 받습니다. sigma는 스케일링 요소에 추가되는 랜덤 노이즈의 양을 제어합니다. 기본값으로 sigma는 0.1로 설정됩니다.

scalingFactor = np.random.normal(loc=1.0, scale=sigma, size=(1, X.shape[1])): 이 코드는 평균이 1.0이고 표준 편차가 sigma인 정규 분포에서 랜덤 샘플을 생성합니다. size 매개변수는 결과 배열의 모양을 지정하는데, (1, X.shape[1])로 설정되어 X의 특성 수와 일치합니다.

myNoise = np.matmul(np.ones((X.shape[0], 1)), scalingFactor): 이 코드는 일련의 1로 구성된 열 벡터와 스케일링 요소 배열을 곱하여 myNoise라는 배열을 생성합니다. 결과 배열은 X와 같은 행 수를 가지며 각 샘플에 대해 스케일링 요소를 복제합니다.

return X * myNoise: 이 코드는 입력 행렬 X를 myNoise 배열과 요소별 곱셈을 통해 스케일링합니다. X의 각 요소는 myNoise에 해당하는 스케일링 요소와 곱해집니다.

이 함수는 전체적으로 입력 행렬 X의 각 특성에 랜덤 스케일링을 적용하며, 스케일링 정도는 sigma 매개변수로 제어됩니다. '''



In [ ]:

#2. shift function   # shift라는 함수를 정의. 주어진 시그널 sig에 시간적인 쉬프트 적용.

def shift(sig, interval=20):
    for col in range(sig.shape[1]):
        offset = np.random.choice(range(-interval, interval))
        sig[:, col] += offset / 1000 
    return sig

 ''' 

각 열(column)에 대해 랜덤한 오프셋(offset)을 생성하여 해당 시그널을 이동시킵니다. 오프셋은 -interval부터 interval까지의 범위에서 랜덤하게 선택됩니다.

# 코드 구성 살펴보기

shift(sig, interval=20): 이 코드는 shift라는 함수를 정의합니다. 이 함수는 sig라는 입력 시그널과 선택적 매개변수인 interval을 받습니다. interval은 오프셋의 범위를 제어하는 매개변수로 기본값은 20입니다.

for col in range(sig.shape[1]):: 이 코드는 sig의 각 열에 대해 반복합니다. sig.shape[1]은 sig의 열의 수를 나타냅니다.

offset = np.random.choice(range(-interval, interval)): 이 코드는 -interval부터 interval까지의 범위에서 랜덤하게 오프셋을 선택합니다. np.random.choice 함수를 사용하여 범위에서 랜덤한 값을 선택합니다.

sig[:, col] += offset / 1000: 이 코드는 sig의 현재 열에 대해 선택된 오프셋을 적용합니다. sig[:, col]은 현재 열의 모든 행을 선택하는 슬라이싱 연산입니다. offset / 1000은 오프셋을 1000으로 나누어 시그널을 이동시킵니다. 이를 통해 시간 단위의 이동을 실수 단위로 나타냅니다.

return sig: 이 코드는 시간적인 쉬프트가 적용된 시그널 sig를 반환합니다.

이 함수는 입력 시그널 sig의 각 열에 대해 랜덤한 시간적인 쉬프트를 적용합니다. 쉬프트의 범위는 interval 매개변수로 제어됩니다.


'''
    

In [ ]:
 #  transform이라는 함수를 정의
 #3. transform function
def transform(sig, train=False):
    if train:
        if np.random.randn() > 0.5: sig = scaling(sig)
        if np.random.randn() > 0.5: sig = shift(sig)
    return sig

''' 
이 함수는 주어진 시그널 sig에 변환을 적용합니다. train 매개변수가 True인 경우에만 변환을 수행하며, 스케일링과 시프트 두 가지 변환이 랜덤하게 적용됩니다.

위 코드를 구성하는 각 부분을 살펴보겠습니다:

transform(sig, train=False): 이 코드는 transform이라는 함수를 정의합니다. 이 함수는 sig라는 입력 시그널과 선택적 매개변수인 train을 받습니다. train은 변환을 수행할지 여부를 제어하는 매개변수로 기본값은 False입니다.

if train:: 이 코드는 train 매개변수가 True인 경우에만 실행되는 조건문입니다. 이 조건문은 변환을 수행할지 여부를 확인합니다.

if np.random.randn() > 0.5:: 이 코드는 랜덤하게 생성된 표준 정규 분포에서 샘플을 추출하여 0.5보다 큰지 여부를 확인합니다. 이를 통해 스케일링 변환을 랜덤하게 선택합니다.

sig = scaling(sig): 이 코드는 scaling 함수를 호출하여 시그널 sig에 스케일링 변환을 적용합니다.

if np.random.randn() > 0.5:: 이 코드는 위와 마찬가지로 랜덤하게 생성된 표준 정규 분포에서 샘플을 추출하여 0.5보다 큰지 여부를 확인합니다. 이를 통해 시프트 변환을 랜덤하게 선택합니다.

sig = shift(sig): 이 코드는 shift 함수를 호출하여 시그널 sig에 시프트 변환을 적용합니다.

return sig: 이 코드는 변환된 시그널 sig를 반환합니다.

이 함수는 주어진 시그널 sig에 대해 훈련(train) 여부에 따라 스케일링 변환과 시프트 변환을 랜덤하게 적용합니다. 변환은 train 매개변수가 True인 경우에만 수행되며, 변환의 선택은 랜덤하게 이루어집니다.

'''

In [ ]:


#4. class ECGDataset
class ECGDataset(Dataset):
    def __init__(self, phase, data_dir, label_csv, folds, leads):
        super(ECGDataset, self).__init__()
        self.phase = phase
        df = pd.read_csv(label_csv)
        df = df[df['fold'].isin(folds)]
        self.data_dir = data_dir
        self.labels = df
        self.leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
        if leads == 'all':
            self.use_leads = np.where(np.in1d(self.leads, self.leads))[0]
        else:
            self.use_leads = np.where(np.in1d(self.leads, leads))[0]
        self.nleads = len(self.use_leads)
        self.classes = ['SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']
        self.n_classes = len(self.classes)
        self.data_dict = {}
        self.label_dict = {}

''' 위의 코드는 ECGDataset이라는 클래스를 정의합니다. 이 클래스는 데이터셋을 나타내며, PyTorch의 Dataset 클래스를 상속합니다.

위 코드를 구성하는 각 부분을 살펴보겠습니다:

def __init__(self, phase, data_dir, label_csv, folds, leads): 이 코드는 ECGDataset 클래스의 생성자(constructor)를 정의합니다. 생성자는 다음 매개변수를 입력으로 받습니다: phase, data_dir, label_csv, folds, leads.

super(ECGDataset, self).__init__(): 이 코드는 Dataset 클래스의 생성자를 호출하여 상속받은 클래스의 생성자를 실행합니다.

self.phase = phase: 이 코드는 인스턴스 변수 phase에 생성자에서 받은 phase 값을 할당합니다. 이 변수는 데이터셋의 단계(예: 훈련, 검증, 테스트)를 나타냅니다.

df = pd.read_csv(label_csv): 이 코드는 CSV 파일인 label_csv를 읽어 데이터프레임 df로 저장합니다. pandas 라이브러리의 read_csv 함수를 사용합니다.

df = df[df['fold'].isin(folds)]: 이 코드는 데이터프레임 df에서 'fold' 열의 값이 folds에 속하는 행만 선택하여 업데이트합니다. isin 함수를 사용하여 특정 값들을 포함하는 행을 선택합니다.

self.data_dir = data_dir: 이 코드는 인스턴스 변수 data_dir에 생성자에서 받은 data_dir 값을 할당합니다. 이 변수는 데이터가 저장된 디렉토리를 나타냅니다.

self.labels = df: 이 코드는 데이터프레임 df를 인스턴스 변수 labels에 할당합니다. 이 변수는 데이터셋의 레이블을 나타냅니다.

self.leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']: 이 코드는 인스턴스 변수 leads에 ECG 신호의 리드(lead)를 나타내는 문자열 목록을 할당합니다.

if leads == 'all':: 이 코드는 leads가 'all'인 경우에 실행되는 조건문입니다. 'all'은 모든 리드를 사용하는 의미를 가집니다.

self.use_leads = np.where(np.in1d(self.leads, self.leads))[0]: 이 코드는 모든 리드를 사용할 경우, self.leads와 동일한 인덱스를 선택하여 self.use_leads에 할당합니다. np.in1d 함수를 사용하여 self.leads의 각 원소가 self.leads와 동일한지 여부를 확인하고, np.where 함수를 사용하여 True인 원소의 인덱스를 추출합니다.

else: self.use_leads = np.where(np.in1d(self.leads, leads))[0]: 이 코드는 leads가 'all'이 아닌 경우에 실행되는 조건문입니다. 선택된 리드만 사용하는 경우, self.leads에서 leads에 포함된 리드만 선택하여 self.use_leads에 할당합니다.

self.nleads = len(self.use_leads): 이 코드는 인스턴스 변수 nleads에 선택된 리드의 수를 할당합니다.

self.classes = ['SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']: 이 코드는 인스턴스 변수 classes에 ECG 클래스를 나타내는 문자열 목록을 할당합니다.

self.n_classes = len(self.classes): 이 코드는 인스턴스 변수 n_classes에 클래스의 수를 할당합니다.

self.data_dict = {}: 이 코드는 빈 딕셔너리를 인스턴스 변수 data_dict에 할당합니다. 이 변수는 데이터를 저장하기 위한 딕셔너리입니다.

self.label_dict = {}: 이 코드는 빈 딕셔너리를 인스턴스 변수 label_dict에 할당합니다. 이 변수는 레이블을 저장하기 위한 딕셔너리입니다.

이 클래스는 ECG 데이터셋을 관리하기 위한 기능을 제공합니다. 데이터셋의 단계, 데이터 디렉토리, 레이블 파일, 선택된 리드, 클래스 등의 정보를 저장하고, 데이터와 레이블을 딕셔너리에 저장합니다.

'''


In [ ]:

#5. getitem function

    def __getitem__(self, index: int):
        row = self.labels.iloc[index]
        patient_id = row['patient_id']
        ecg_data, _ = wfdb.rdsamp(os.path.join(self.data_dir, patient_id))
        ecg_data = transform(ecg_data, self.phase == 'train')
        nsteps, _ = ecg_data.shape
        ecg_data = ecg_data[-15000:, self.use_leads]
        result = np.zeros((15000, self.nleads)) # 30 s, 500 Hz
        result[-nsteps:, :] = ecg_data
        if self.label_dict.get(patient_id):
            labels = self.label_dict.get(patient_id)
        else:
            labels = row[self.classes].to_numpy(dtype=np.float32)
            self.label_dict[patient_id] = labels
        return torch.from_numpy(result.transpose()).float(), torch.from_numpy(labels).float()

    def __len__(self):
        return len(self.labels)


''' 위의 코드는 __getitem__과 __len__ 메서드를 정의합니다. 이 메서드들은 데이터셋에서 특정 인덱스의 샘플을 가져오고 데이터셋의 길이를 반환하는 역할을 합니다.

__getitem__(self, index: int): 이 코드는 데이터셋에서 주어진 인덱스에 해당하는 샘플을 가져오는 메서드입니다. index는 가져올 샘플의 인덱스를 나타냅니다.

row = self.labels.iloc[index]: 이 코드는 데이터프레임 labels에서 index에 해당하는 행을 선택하여 row에 저장합니다.

patient_id = row['patient_id']: 이 코드는 row에서 'patient_id' 열의 값을 가져와 patient_id에 저장합니다. 이는 해당 샘플의 환자 ID를 나타냅니다.

ecg_data, _ = wfdb.rdsamp(os.path.join(self.data_dir, patient_id)): 이 코드는 wfdb.rdsamp 함수를 사용하여 ECG 데이터를 로드합니다. 데이터는 self.data_dir 디렉토리의 patient_id 파일에서 읽어옵니다. rdsamp 함수는 ECG 신호 데이터와 추가 정보를 반환합니다. 여기서는 ECG 신호 데이터만 ecg_data에 저장하고, 추가 정보는 무시합니다.

ecg_data = transform(ecg_data, self.phase == 'train'): 이 코드는 transform 함수를 사용하여 ECG 데이터에 변환을 적용합니다. 변환은 self.phase가 'train'일 때만 적용됩니다.

nsteps, _ = ecg_data.shape: 이 코드는 ECG 데이터의 형태(shape)에서 시간 스텝 수와 리드 수를 가져와 nsteps에 저장합니다.

ecg_data = ecg_data[-15000:, self.use_leads]: 이 코드는 ECG 데이터에서 마지막 15000개의 시간 스텝과 선택된 리드만 선택하여 ecg_data에 저장합니다.

result = np.zeros((15000, self.nleads)): 이 코드는 15000개의 시간 스텝과 선택된 리드 수로 이루어진 0으로 채워진 배열 result를 생성합니다.

result[-nsteps:, :] = ecg_data: 이 코드는 result 배열의 마지막 nsteps 행에 ecg_data 값을 할당합니다. 이를 통해 result 배열에 ECG 데이터를 채웁니다.

if self.label_dict.get(patient_id):: 이 코드는 label_dict 딕셔너리에서 patient_id를 키로 가지는 값이 있는지 확인합니다.

labels = self.label_dict.get(patient_id): 이 코드는 label_dict 딕셔너리에서 patient_id를 키로 가지는 값을 가져옵니다.

else: labels = row[self.classes].to_numpy(dtype=np.float32): 이 코드는 label_dict 딕셔너리에 patient_id를 키로 가지는 값이 없는 경우, row에서 self.classes에 해당하는 열을 선택하여 배열로 변환한 후 labels에 저장합니다.

self.label_dict[patient_id] = labels: 이 코드는 label_dict 딕셔너리에 patient_id를 키로 하고 labels를 값으로 하는 항목을 추가합니다. 이를 통해 이후 동일한 patient_id에 대해서는 다시 계산할 필요 없이 저장된 레이블을 사용할 수 있습니다.

return torch.from_numpy(result.transpose()).float(), torch.from_numpy(labels).float(): 이 코드는 result 배열을 텐서로 변환하여 첫 번째 반환값으로 반환하고, labels 배열을 텐서로 변환하여 두 번째 반환값으로 반환합니다. 반환되는 텐서들은 float 자료형으로 변환됩니다.

__len__(self): 이 코드는 데이터셋의 샘플 수를 반환하는 메서드입니다. len(self.labels)를 반환하여 데이터프레임 labels의 길이를 반환합니다.


'''
    